# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.75,74,38,12.08,TF,1735511624
1,1,blackmans bay,-43.0167,147.3167,19.85,61,87,2.14,AU,1735511626
2,2,vilyuchinsk,52.9306,158.4028,-13.85,82,78,5.20,RU,1735511627
3,3,adamstown,-25.0660,-130.1015,24.30,84,100,7.92,PN,1735511628
4,4,ushuaia,-54.8000,-68.3000,18.81,39,0,2.57,AR,1735511630


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    'Lng', 
    'Lat',  
    geo=True,  
    size='Humidity',  
)
# Display the map
city_humidity_map

:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_cleaned_df = city_data_df.dropna()

# Drop any rows with null values
ideal_weather_df = city_cleaned_df[(city_cleaned_df['Max Temp'] >= 15.6) & (city_cleaned_df['Max Temp'] <= 29.4) & (city_cleaned_df['Humidity'] < 40)]

# Display sample data
ideal_weather_df.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,ushuaia,-54.8000,-68.3000,18.81,39,0,2.57,AR,1735511630
28,28,la junta,37.9850,-103.5438,17.16,29,0,5.14,US,1735511661
40,40,alice springs,-23.7000,133.8833,26.75,21,0,0.00,AU,1735511678
42,42,akjoujt,19.6969,-14.4164,24.10,22,82,3.59,MR,1735511680
90,90,broken hill,-31.9500,141.4333,26.25,22,2,3.98,AU,1735511745
128,128,faya-largeau,17.9257,19.1043,18.42,22,65,10.26,TD,1735511797
156,156,kidal,18.4411,1.4078,21.16,14,89,4.02,ML,1735511833
183,183,ouadda,8.0777,22.4007,25.46,18,1,4.61,CF,1735511875
242,242,talodi,10.6325,30.3797,23.78,23,4,3.94,SD,1735511956
244,244,presidente epitacio,-21.7633,-52.1156,27.49,36,0,1.38,BR,1735511959


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,ushuaia,AR,-54.8000,-68.3000,39,
28,la junta,US,37.9850,-103.5438,29,
40,alice springs,AU,-23.7000,133.8833,21,
42,akjoujt,MR,19.6969,-14.4164,22,
90,broken hill,AU,-31.9500,141.4333,22,
128,faya-largeau,TD,17.9257,19.1043,22,
156,kidal,ML,18.4411,1.4078,14,
183,ouadda,CF,8.0777,22.4007,18,
242,talodi,SD,10.6325,30.3797,23,
244,presidente epitacio,BR,-21.7633,-52.1156,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
category = "accommodation.hotel"
radius = 10000  
limit = 1
params = {"categories": category, "limit": limit, "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
la junta - nearest hotel: Travel Inn of La Junta
alice springs - nearest hotel: Aurora Alice Springs
akjoujt - nearest hotel: Hotel Fatima فندق فاطمة
broken hill - nearest hotel: Ibis Styles
faya-largeau - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
ouadda - nearest hotel: No hotel found
talodi - nearest hotel: No hotel found
presidente epitacio - nearest hotel: Pousada Orla do Sol
bardai - nearest hotel: No hotel found
deming - nearest hotel: No hotel found
massenya - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
tonj - nearest hotel: No hotel found
perryton - nearest hotel: Ambassador Inn
sao vicente - nearest hotel: Pousada Vitória
bouna - nearest hotel: Hôtel Éléphant  Bouna
esquel - nearest hotel: El Quincho
yumbe - nearest hotel: Bayan Yumbe Hotel
tidjikja - nearest hotel: No hotel found
bee cave - nearest hotel: Sone

,City,Country,Lat,Lng,Humidity,Hotel Name
4,ushuaia,AR,-54.8000,-68.3000,39,Apart Hotel Aires del Beagle
28,la junta,US,37.9850,-103.5438,29,Travel Inn of La Junta
40,alice springs,AU,-23.7000,133.8833,21,Aurora Alice Springs
42,akjoujt,MR,19.6969,-14.4164,22,Hotel Fatima فندق فاطمة
90,broken hill,AU,-31.9500,141.4333,22,Ibis Styles
128,faya-largeau,TD,17.9257,19.1043,22,No hotel found
156,kidal,ML,18.4411,1.4078,14,No hotel found
183,ouadda,CF,8.0777,22.4007,18,No hotel found
242,talodi,SD,10.6325,30.3797,23,No hotel found
244,presidente epitacio,BR,-21.7633,-52.1156,36,Pousada Orla do Sol


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM',
    frame_width = 1000, 
    frame_height = 500, 
    color = 'City', 
    size = 'Humidity',
    scale = 1.2,
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)